# Configurable Heatmap (Voilá/Binder/JupyterLite)
This notebook opens as an interactive app. Use **Configure** then **Rate & Generate**.


In [ ]:
# MULTI-CATEGORY HEATMAP — CONFIGURABLE QUESTIONS + SIMPLE 1/3/5 RATINGS
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import json, re
from urllib.parse import quote

DEFAULT_CONFIG = {
    "Spend Visibility & Control": [
        "Q1: Consolidated real-time view of total spend (Qualitative)",
        "Q2: Spend Under Management % (KPI, Best ≥80%)",
        "Q3: Maverick Spend % (KPI, Best ≤5%)",
        "Q4: Detailed categorization & consistent tagging (Qualitative)",
        "Q5: Contract Compliance % (KPI, Best ≥90%)",
    ],
    "Vendor & Supply Chain Mgmt": [
        "Q1: Top-10 supplier concentration & dual/multi-sourcing (KPI/Qual)",
        "Q2: OTIF – On-Time, In-Full % (KPI)",
        "Q3: Supplier performance reviews tied to C/Q/D & action plans (Qual)",
        "Q4: % spend under contracts with cost-reduction clauses/value-add (KPI)",
        "Q5: Total Landed Cost used in sourcing/renewals (Qual)",
    ],
    "Process & Workforce Efficiency": [
        "Q1: Touchless/straight-through processing rate (KPI)",
        "Q2: Cost per transaction (AP invoice, service ticket, etc.) (KPI)",
        "Q3: First-pass yield / rework rate (KPI)",
        "Q4: Cycle time for key processes (P2P, O2C, monthly close) (KPI)",
        "Q5: Workforce utilization & overtime control (Qual/KPI)",
    ],
    "Production & Asset Utilization": [
        "Q1: OEE – Overall Equipment Effectiveness % (KPI)",
        "Q2: Unplanned downtime as % of scheduled time (KPI)",
        "Q3: % assets under preventive/predictive maintenance (KPI)",
        "Q4: Inventory health (Turns or Days on Hand) (KPI)",
        "Q5: Capacity utilization vs plan (KPI/Qual)",
    ],
    "Energy & Facility Costs": [
        "Q1: Energy intensity (kWh per output unit/per sqft) tracked & improved (KPI)",
        "Q2: % energy spend under hedged/contracted rates (KPI)",
        "Q3: Facility occupancy/utilization rate (KPI)",
        "Q4: Data center PUE / IT energy KPI (KPI)",
        "Q5: Waste/water cost control & recycling/reuse programs (Qual/KPI)",
    ],
}

COLUMNS = ["Procurement", "Finance", "IT", "Operations"]
SCORES  = [1, 3, 5]
COLOR_MAP = {1: "#FF9999", 3: "#FFF59D", 5: "#A5D6A7"}
Q_COL_W = "520px"; CELL_W = "90px"; CELL_H = "36px"
FONT = "system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial, sans-serif"

config = json.loads(json.dumps(DEFAULT_CONFIG))
dropdowns = {}

def sanitize_filename(name: str) -> str:
    import re
    safe = re.sub(r'[^A-Za-z0-9_\-]+', '_', name.strip())
    return safe.strip("_") or "heatmap"

def render_html_table(questions, rows, columns, title):
    css = f"""
    <style>
      .hm-wrap {{ font-family:{FONT}; color:#1f2937; }}
      .hm-title {{ font-weight:700; margin:0 0 6px 0; }}
      .hm-table {{ border-collapse: collapse; border:1px solid #d1d5db; width: fit-content; font-size:13px; }}
      .hm-table th, .hm-table td {{ border:1px solid #e5e7eb; width:{CELL_W}; height:{CELL_H}; text-align:center; vertical-align:middle; padding:0; }}
      .hm-table th:first-child, .hm-table td:first-child {{ width:{Q_COL_W}; text-align:left; padding:6px 10px; white-space:normal; line-height:1.3; }}
      .hm-table th {{ background:#f7f7f7; font-weight:600; }}
    </style>
    """
    html = [f'<div class="hm-wrap">', f'<div class="hm-title">{title}</div>', '<table class="hm-table">']
    html.append('<tr>'); html.append('<th>Questions</th>')
    for c in columns: html.append(f'<th>{c}</th>')
    html.append('</tr>')
    for i, q in enumerate(questions):
        html.append('<tr>'); html.append(f'<td title="{q}">{q}</td>')
        for val in rows[i]:
            try: v = int(val)
            except Exception: v = 3
            bg = COLOR_MAP.get(v, "#ffffff")
            html.append(f'<td style="background:{bg};">{v}</td>')
        html.append('</tr>')
    html.append('</table></div>')
    return css + "\n".join(html)

def collect_scores_for_category(cat):
    qs = list(config.get(cat, [])); matrix = []
    for q in qs:
        row = []
        for col in COLUMNS:
            w = dropdowns.get(cat, {}).get((q, col))
            row.append(int(w.value) if w else 3)
        matrix.append(row)
    return qs, matrix

def rebuild_dropdowns_for_category(cat):
    old = dropdowns.get(cat, {}); new = {}
    for q in config.get(cat, []):
        for col in COLUMNS:
            key = (q, col); prev = old.get(key); val = prev.value if prev else 3
            new[key] = widgets.Dropdown(options=SCORES, value=val, layout=widgets.Layout(width=CELL_W, height=CELL_H))
    dropdowns[cat] = new

def build_grid_widget(cat):
    if cat not in dropdowns: rebuild_dropdowns_for_category(cat)
    header_cells = [widgets.HTML("<b>Questions</b>", layout=widgets.Layout(width=Q_COL_W))] + \
                   [widgets.HTML(f"<b>{c}</b>", layout=widgets.Layout(width=CELL_W)) for c in COLUMNS]
    rows = [widgets.HBox(header_cells)]
    for q in config.get(cat, []):
        row_dds = [dropdowns[cat][(q, c)] for c in COLUMNS]
        rows.append(widgets.HBox([widgets.Label(q, layout=widgets.Layout(width=Q_COL_W))] + row_dds))
    return widgets.VBox(rows, layout=widgets.Layout(gap="6px"))

def csv_text_for_category(cat, qs, matrix):
    lines = []; header = ["Questions"] + COLUMNS
    lines.append(",".join([f'"{h}"' if "," in h or " " in h else h for h in header]))
    for q, row in zip(qs, matrix):
        q_escaped = q.replace('"', '""')
        line = [f'"{q_escaped}"'] + [str(int(v)) for v in row]
        lines.append(",".join(line))
    return "\n".join(lines)

def make_download_link(filename, text, label):
    href = f"data:text/csv;charset=utf-8,{quote(text)}"
    return f'<a download="{filename}" href="{href}">{label}</a>'

cat_select = widgets.Dropdown(options=list(config.keys()), description="Category:", layout=widgets.Layout(width="620px"))
generate_btn     = widgets.Button(description="Generate Category Heatmap", button_style="primary")
generate_all_btn = widgets.Button(description="Generate All", button_style="")
export_btn       = widgets.Button(description="Export CSVs", button_style="")
output_area = widgets.Output()
grid_box    = build_grid_widget(cat_select.value)
controls_rg = widgets.HBox([cat_select, generate_btn, generate_all_btn, export_btn], layout=widgets.Layout(gap="8px"))
rate_generate_box = widgets.VBox([controls_rg, grid_box, output_area])

def on_cat_change(change):
    if change["name"] == "value":
        with output_area: clear_output()
        rg_container.children = [controls_rg, build_grid_widget(change["new"]), output_area]
cat_select.observe(on_cat_change, names="value")

def on_generate(_):
    cat = cat_select.value
    qs, matrix = collect_scores_for_category(cat)
    html = render_html_table(qs, matrix, COLUMNS, title=cat)
    with output_area:
        clear_output(); display(HTML(html))

def on_generate_all(_):
    with output_area:
        clear_output()
        for cat in config.keys():
            qs, matrix = collect_scores_for_category(cat)
            display(HTML(render_html_table(qs, matrix, COLUMNS, title=cat)))
            display(HTML("<div style='height:8px;'></div>"))

def on_export(_):
    blocks = []
    for cat in config.keys():
        qs, matrix = collect_scores_for_category(cat)
        csv_txt = csv_text_for_category(cat, qs, matrix)
        fname = f"heatmap_{sanitize_filename(cat)}.csv"
        link = make_download_link(fname, csv_txt, f"Download CSV – {cat}")
        blocks.append(f"<div style='margin:4px 0'>{link}</div>")
    with output_area:
        display(HTML("<hr style='margin:10px 0'>"))
        display(HTML("<b>Exports:</b>")); display(HTML("".join(blocks)))

generate_btn.on_click(on_generate)
generate_all_btn.on_click(on_generate_all)
export_btn.on_click(on_export)
rg_container = widgets.VBox([controls_rg, grid_box, output_area])

cfg_output = widgets.Output()
cfg_cat_select = widgets.Dropdown(options=list(config.keys()), description="Category:", layout=widgets.Layout(width="580px"))
new_q_text   = widgets.Text(placeholder="Type a new question…", layout=widgets.Layout(width="580px"))
add_q_btn    = widgets.Button(description="Add Question", button_style="success")
rem_q_select = widgets.Dropdown(options=[], description="Remove:", layout=widgets.Layout(width="580px"))
rem_q_btn    = widgets.Button(description="Remove Selected Question", button_style="danger")
rename_q_select = widgets.Dropdown(options=[], description="Rename:", layout=widgets.Layout(width="580px"))
rename_q_text   = widgets.Text(placeholder="New question text…", layout=widgets.Layout(width="580px"))
rename_q_btn    = widgets.Button(description="Apply Rename", button_style="warning")

def refresh_question_selectors(cat=None):
    cat = cat or cfg_cat_select.value
    qs = config.get(cat, [])
    rem_q_select.options = qs; rename_q_select.options = qs
refresh_question_selectors()

def on_cfg_cat_change(change):
    if change["name"] == "value":
        refresh_question_selectors(change["new"])
cfg_cat_select.observe(on_cfg_cat_change, names="value")

def on_add_q(_):
    cat = cfg_cat_select.value; txt = new_q_text.value.strip()
    if not txt:
        with cfg_output: clear_output(); print("Please enter a question to add."); return
    config[cat].append(txt); rebuild_dropdowns_for_category(cat); refresh_question_selectors(cat); new_q_text.value = ""
    if cat_select.value == cat:
        rg_container.children = [controls_rg, build_grid_widget(cat), output_area]
        with output_area: clear_output()
    with cfg_output: clear_output(); print(f"Added question to '{cat}'.")

def on_rem_q(_):
    cat = cfg_cat_select.value; sel = rem_q_select.value
    if sel is None or sel not in config.get(cat, []):
        with cfg_output: clear_output(); print("Select a question to remove."); return
    config[cat] = [q for q in config[cat] if q != sel]
    rebuild_dropdowns_for_category(cat); refresh_question_selectors(cat)
    if cat_select.value == cat:
        rg_container.children = [controls_rg, build_grid_widget(cat), output_area]
        with output_area: clear_output()
    with cfg_output: clear_output(); print(f"Removed question from '{cat}'.")

def on_rename_q(_):
    cat = cfg_cat_select.value; old = rename_q_select.value; new = rename_q_text.value.strip()
    if not old or not new:
        with cfg_output: clear_output(); print("Pick a question and enter a new name."); return
    qs = config.get(cat, [])
    try: idx = qs.index(old)
    except ValueError:
        with cfg_output: clear_output(); print("Selected question no longer exists."); return
    qs[idx] = new; config[cat] = qs; rebuild_dropdowns_for_category(cat); refresh_question_selectors(cat); rename_q_text.value = ""
    if cat_select.value == cat:
        rg_container.children = [controls_rg, build_grid_widget(cat), output_area]
        with output_area: clear_output()
    with cfg_output: clear_output(); print(f"Renamed question in '{cat}'.")

add_q_btn.on_click(on_add_q); rem_q_btn.on_click(on_rem_q); rename_q_btn.on_click(on_rename_q)

json_text = widgets.Textarea(value=json.dumps(config, indent=2), description="Config JSON:", layout=widgets.Layout(width="700px", height="250px"))
apply_json_btn    = widgets.Button(description="Apply JSON to Config", button_style="primary")
refresh_json_btn  = widgets.Button(description="Refresh JSON from Current", button_style="")
download_json_btn = widgets.Button(description="Download Config JSON", button_style="")
json_links_out = widgets.Output()

def on_apply_json(_):
    txt = json_text.value.strip()
    try:
        obj = json.loads(txt)
        for k, v in obj.items():
            if not isinstance(k, str) or not isinstance(v, list) or not all(isinstance(x, str) for x in v):
                raise ValueError("Config must be { 'Category': ['Q1', 'Q2', ...], ... }")
    except Exception as e:
        with cfg_output: clear_output(); print("Invalid JSON config:", e); return
    global config; config = obj
    for cat in config.keys(): rebuild_dropdowns_for_category(cat)
    cat_select.options = list(config.keys()); cfg_cat_select.options = list(config.keys())
    refresh_question_selectors(); json_text.value = json.dumps(config, indent=2)
    rg_container.children = [controls_rg, build_grid_widget(cat_select.value), output_area]
    with cfg_output: clear_output(); print("Applied JSON and rebuilt UI.")

def on_refresh_json(_):
    json_text.value = json.dumps(config, indent=2)
    with cfg_output: clear_output(); print("Refreshed JSON from current config.")

def on_download_json(_):
    with json_links_out:
        json_links_out.clear_output()
        blob = json.dumps(config, indent=2)
        fname = f"heatmap_config.json"
        link = f'<a download="{fname}" href="data:application/json;charset=utf-8,{quote(blob)}">Download Config JSON</a>'
        display(HTML(link))

apply_json_btn.on_click(on_apply_json); refresh_json_btn.on_click(on_refresh_json); download_json_btn.on_click(on_download_json)

cfg_quick_box = widgets.VBox([
    cfg_cat_select,
    widgets.HBox([new_q_text, add_q_btn]),
    widgets.HBox([rem_q_select, rem_q_btn]),
    widgets.HBox([rename_q_select, rename_q_text, rename_q_btn]),
    cfg_output
], layout=widgets.Layout(gap="6px"))

cfg_json_box = widgets.VBox([
    json_text,
    widgets.HBox([apply_json_btn, refresh_json_btn, download_json_btn], layout=widgets.Layout(gap="8px")),
    json_links_out
])

configure_box = widgets.VBox([
    widgets.HTML("<h4 style='margin:0 0 8px 0'>Configure Categories & Questions</h4>"),
    cfg_quick_box,
    widgets.HTML("<hr style='margin:10px 0'>"),
    widgets.HTML("<h4 style='margin:0 0 8px 0'>Load/Save via JSON</h4>"),
    cfg_json_box,
])

tabs = widgets.Tab(children=[configure_box, rg_container])
tabs.set_title(0, "Configure"); tabs.set_title(1, "Rate & Generate")
display(tabs)
with output_area: clear_output()
